In [2]:
import numpy as np
import pandas as pd

import matplotlib
%matplotlib inline

In [3]:
housing = pd.read_csv('../datasets/housing/housing.csv')

In [4]:
housing.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


# 1. Try a Support Vector Machine Regressor

In [5]:
tmp = housing.copy()

Add the extra features:

In [6]:
tmp["rooms_per_household"]       = tmp["total_rooms"] / tmp["households"]
tmp["bedrooms_per_room"]         = tmp["total_bedrooms"] / tmp["total_rooms"]
tmp["population_per_household"]  = tmp["population"] / tmp["households"]
tmp['income_cat']                = np.ceil(tmp['median_income'].div(1.5)).clip(1,5)

Add one-hot encoding:

In [8]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder()
encoded = encoder.fit_transform(tmp['ocean_proximity'].values[:, None])

In [9]:
tmp = pd.concat([
    tmp.drop(columns='ocean_proximity'),
    pd.DataFrame(encoded.toarray(), columns=np.ravel(encoder.categories_))
], axis=1)

In [10]:
tmp.corr()['median_house_value'].sort_values(ascending=False)

median_house_value          1.000000
median_income               0.688075
income_cat                  0.643892
<1H OCEAN                   0.256617
NEAR BAY                    0.160284
rooms_per_household         0.151948
NEAR OCEAN                  0.141862
total_rooms                 0.134153
housing_median_age          0.105623
households                  0.065843
total_bedrooms              0.049686
ISLAND                      0.023416
population_per_household   -0.023737
population                 -0.024650
longitude                  -0.045967
latitude                   -0.144160
bedrooms_per_room          -0.255880
INLAND                     -0.484859
Name: median_house_value, dtype: float64

Split the data into train and test sets

In [23]:
from sklearn.model_selection import StratifiedShuffleSplit

splitter = StratifiedShuffleSplit(n_splits=1, train_size=0.8, random_state=42)
for idx_train, idx_test in splitter.split(tmp, tmp['income_cat']):
    train = tmp.loc[idx_train]
    test = tmp.loc[idx_test]
    
label = train['median_house_value']
train = train.drop(columns='median_house_value')

In [32]:
train

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,rooms_per_household,bedrooms_per_room,population_per_household,income_cat,<1H OCEAN,INLAND,ISLAND,NEAR BAY,NEAR OCEAN
17606,-121.89,37.29,38.0,1568.0,351.0,710.0,339.0,2.7042,4.625369,0.223852,2.094395,2.0,1.0,0.0,0.0,0.0,0.0
18632,-121.93,37.05,14.0,679.0,108.0,306.0,113.0,6.4214,6.008850,0.159057,2.707965,5.0,1.0,0.0,0.0,0.0,0.0
14650,-117.20,32.77,31.0,1952.0,471.0,936.0,462.0,2.8621,4.225108,0.241291,2.025974,2.0,0.0,0.0,0.0,0.0,1.0
3230,-119.61,36.31,25.0,1847.0,371.0,1460.0,353.0,1.8839,5.232295,0.200866,4.135977,2.0,0.0,1.0,0.0,0.0,0.0
3555,-118.59,34.23,17.0,6592.0,1525.0,4459.0,1463.0,3.0347,4.505810,0.231341,3.047847,3.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6563,-118.13,34.20,46.0,1271.0,236.0,573.0,210.0,4.9312,6.052381,0.185681,2.728571,4.0,0.0,1.0,0.0,0.0,0.0
12053,-117.56,33.88,40.0,1196.0,294.0,1052.0,258.0,2.0682,4.635659,0.245819,4.077519,2.0,0.0,1.0,0.0,0.0,0.0
13908,-116.40,34.09,9.0,4855.0,872.0,2098.0,765.0,3.2723,6.346405,0.179609,2.742484,3.0,0.0,1.0,0.0,0.0,0.0
11159,-118.01,33.82,31.0,1960.0,380.0,1356.0,356.0,4.0625,5.505618,0.193878,3.808989,3.0,1.0,0.0,0.0,0.0,0.0


In [31]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer('median')
imputer.fit_transform(train)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [20]:
from sklearn.preprocessing import StandardScaler

columns = train.columns.to_series()['longitude':'income_cat']
train = StandardScaler().fit_transform(train[columns])

In [22]:
from sklearn.linear_model import LinearRegression

regressor = LinearRegression()
regressor.fit(train, label)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').